In [1]:
import sys 
sys.path.append('../pydss')
import tools
import solvers
import numpy as np 
import sympy as sym
from sympy.utilities.lambdify import lambdify, implemented_function
from imp import reload
import numba 
sin = sym.sin
cos = sym.cos

## Symbolic variable definitions

In [2]:
delta, omega, e1d, e1q = sym.symbols('delta omega e1d e1q')
i_d, i_q, v_d, v_q = sym.symbols('i_d i_q v_d v_q')
v_f, p_m = sym.symbols('v_f p_m')
X_d, X_q, X1d, X1q, X_l, R_a, T1d0, T1q0  = sym.symbols('X_d X_q X1d X1q X_l R_a T1d0 T1q0')
H, D = sym.symbols('H D')
V_t, theta_t, P_t, Q_t, V_inf, theta_inf = sym.symbols('V_t, theta_t, P_t, Q_t, V_inf, theta_inf')
B_t_inf, B_t0, G_t_inf, G_t0 = sym.symbols('B_t_inf, B_t0, G_t_inf, G_t0')
Omega_b, p_e = sym.symbols('Omega_b p_e')

## Initialization problem (backward)

In [3]:
N_x = 4
N_y = 9
ddelta = Omega_b * (omega - 1);
domega = 1/(2*H) * ( p_m - p_e - D * ( omega - 1)) ;
de1q   = 1/T1d0 * ( -e1q - (X_d - X1d ) * i_d + v_f);
de1d   = 1/T1q0 * ( -e1d + (X_q - X1q ) * i_q );

g1 = v_q + R_a * i_q - e1q + (X1d - X_l)*i_d;
g2 = v_d + R_a * i_d - e1d - (X1q - X_l)*i_q;
g3 = p_e - (v_q + R_a*i_q)*i_q - (v_d + R_a*i_d)*i_d ;
g4 = v_d - V_t * sin(delta - theta_t);
g5 = v_q - V_t * cos(delta - theta_t);
g6 = v_d * i_d + v_q * i_q - P_t; 
g7 = v_q * i_d - v_d * i_q - Q_t;
g8 =  V_t**2*(G_t_inf+G_t0) - V_t*V_inf*(G_t_inf * cos(theta_t - theta_inf) + B_t_inf * sin(theta_t - theta_inf)) - P_t  
g9 = -V_t**2*(B_t_inf+B_t0) - V_t*V_inf*(G_t_inf * sin(theta_t - theta_inf) - B_t_inf * cos(theta_t - theta_inf)) - Q_t  

In [4]:
x = sym.Matrix([delta,omega,e1q,e1d])
f = sym.Matrix([ddelta, domega, de1q, de1d])

y = sym.Matrix([i_d,i_q,p_e,v_d,v_q,p_m,v_f,theta_t,Q_t])
g = sym.Matrix([g1,g2,g3,g4,g5,g6,g7,g8,g9])

In [5]:
F_x = f.jacobian(x)
F_y = f.jacobian(y)
G_x = g.jacobian(x)
G_y = g.jacobian(y)

In [6]:
X_d = 1.81
X1d  = 0.3
T1d0  = 8.0
X_q  = 1.76
X1q  = 0.65
T1q0  = 1.0 
R_a  =  0.003 
X_l  = 0.15  
H  = 3.5   
Omega_b = 2*np.pi*60
Z_t_inf = 0.15+1.0/(1.0/0.5+1.0/0.93)

params = {
'X_d':  X_d,
'X1d':  X1d,
'T1d0': T1d0,
'X_q':  X_q,
'X1q':  X1q,
'T1q0': T1q0,
'R_a':  R_a,
'X_l': X_l,
'H':  H,
'D' : 0.1,
'Omega_b': Omega_b,
'B_t_inf':-1.0/Z_t_inf,
'B_t0':0.0, 
'G_t_inf':0.01,
'G_t0':0.1,
'V_inf':0.90081,
'theta_inf':0.0,   
}

u = sym.Matrix([P_t,V_t])

In [7]:
sym_dict = {'f':f,'g':g,'x':x,'y':y,'u':u,'F_x':F_x,'G_x':G_x,'F_y':F_y,'G_y':G_y,'params':params}
out_file = 'sys2.py'
tools.sym2num(sym_dict,out_file)

In [8]:
import sys2
sys1 = reload(sys2)
s1 = sys2.system()

In [9]:
%timeit sys2.update(s1.struct,0,0)


The slowest run took 649586.52 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 1.67 µs per loop


In [10]:
s1.struct[0]['u'][0] = 0.8
s1.struct[0]['u'][1] = 1.0
s1.struct[0]['x'] = np.ones((N_x,1))
s1.struct[0]['y'] = np.ones((N_y,1))
#%timeit nr(s1.struct)

N_points = 1000
X = np.zeros((N_x,N_points))
it = 0
for p in np.linspace(0.4,1.0,N_points):
    s1.struct[0]['u'][0] = p
    solvers.nr(sys2.update,s1.struct)
    X[:,it] = s1.struct[0]['x'][:,0]
    it+=1

TypingError: Failed at nopython (nopython frontend)
Invalid usage of pyobject with parameters (unaligned array(Record([('N_x', '<i4'), ('N_y', '<i4'), ('x', '<f8', (4, 1)), ('y', '<f8', (9, 1)), ('u', '<f8', (2, 1)), ('f', '<f8', (4, 1)), ('g', '<f8', (9, 1)), ('F_x', '<f8', (4, 4)), ('F_y', '<f8', (4, 9)), ('G_x', '<f8', (9, 4)), ('G_y', '<f8', (9, 9)), ('T1q0', '<f8'), ('X_d', '<f8'), ('X_l', '<f8'), ('G_t_inf', '<f8'), ('Omega_b', '<f8'), ('G_t0', '<f8'), ('V_inf', '<f8'), ('R_a', '<f8'), ('X1q', '<f8'), ('T1d0', '<f8'), ('H', '<f8'), ('B_t_inf', '<f8'), ('theta_inf', '<f8'), ('D', '<f8'), ('X1d', '<f8'), ('X_q', '<f8'), ('B_t0', '<f8')]), 1d, C), int64, int64)
No type info available for pyobject as a callable.
File "../pydss/solvers.py", line 113
[1] During: resolving callee type: pyobject
[2] During: typing of call at ../pydss/solvers.py (113)

This error may have been caused by the following argument(s):
- argument 0: cannot determine Numba type of <class 'numba.targets.registry.

In [14]:
sys2.update(s1.struct,0,0)

In [ ]:
u

In [ ]:
solvers.ssa(s1.struct)

In [ ]:
G_y


In [ ]:
s1.struct[0]['G_y']

## Simulation problem (forward)

In [ ]:
N_x = 4
N_y = 9

x = sym.Matrix([delta,omega,e1q,e1d])
y = sym.Matrix([i_d,i_q,p_e,v_d,v_q,P_t,Q_t,V_t,theta_t])
u = sym.Matrix([p_m,v_f,D])

f = sym.Matrix([ddelta, domega, de1q, de1d])
g = sym.Matrix([g1, g2, g3, g4, g5,g6, g7, g8,g9])

F_x = f.jacobian(x)
F_y = f.jacobian(y)
G_x = g.jacobian(x)
G_y = g.jacobian(y)

In [ ]:
sym_dict = {'f':f,'g':g,'x':x,'y':y,'u':u,'F_x':F_x,'G_x':G_x,'F_y':F_y,'G_y':G_y,'params':params}
out_file = 'sys_sim.py'
tools.sym2num(sym_dict,out_file)

In [ ]:
import sys_sim
sys_sim = reload(sys_sim)
s_sim = sys_sim.system()

In [ ]:
s_sim.struct[0]['u'][0] = 0.5
s_sim.struct[0]['u'][1] = 1.1
s_sim.struct[0]['u'][2] = 0.1
s_sim.struct[0]['x'][0] = 0.0

nr(s_sim.struct)

In [ ]:
s_sim.struct[0]['x'] += 1

In [ ]:
s_sim.struct[0]['x']

In [15]:
@numba.jit(nopython=True,cache=True)
def fun(sin,a):
    return sin(a)



In [16]:
fun(np.sin,2)

TypingError: Failed at nopython (nopython frontend)
Invalid usage of pyobject with parameters (int64)
No type info available for pyobject as a callable.
File "<ipython-input-15-6359a8d40e01>", line 3
[1] During: resolving callee type: pyobject
[2] During: typing of call at <ipython-input-15-6359a8d40e01> (3)

This error may have been caused by the following argument(s):
- argument 0: cannot determine Numba type of <class 'numpy.ufunc'>
